In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score,confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as plo
import pickle
import json

In [3]:
data = pd.read_csv('Data_Arbre.csv')

new_data = ["haut_tot", "haut_tronc", "tronc_diam", "age_estim", "fk_arb_etat",
                "fk_stadedev", "fk_port", "fk_pied", "fk_situation", "fk_revetement","longitude",'latitude']

new_data1 = ["haut_tot", "haut_tronc", "tronc_diam", "age_estim",
                "fk_stadedev", "fk_port", "fk_pied", "fk_situation", "fk_revetement","longitude",'latitude']

data = data[new_data]

data['fk_arb_etat'] = data['fk_arb_etat'].apply(lambda x: 'Non Essouché' if x != 'Essouché' else x)

donnes_qualitatives = ["fk_arb_etat", "fk_stadedev", "fk_port", "fk_pied", "fk_situation", "fk_revetement"]
donnes_quantitatives = ["haut_tot", "haut_tronc", "tronc_diam", "age_estim"]

# 2) on encode nos données :

encoding = OrdinalEncoder()

data[donnes_qualitatives] = encoding.fit_transform(data[donnes_qualitatives])

# 3) Normalisation des données:

std = StandardScaler()

data[donnes_quantitatives] = std.fit_transform(data[donnes_quantitatives])

X = pd.DataFrame(data[donnes_quantitatives])
y = data["fk_arb_etat"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train,X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2)



In [8]:
def evaluation_modele(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    matrice = confusion_matrix(y_true, y_pred)
    print("confusion matrix:", matrice)
    print("accuracy:", accuracy)
    print("f1-score:", f1)


    matrice = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=matrice, display_labels=np.unique(y_true))
    disp.plot()



def gradient():
    resultat = {}
    # regarder avec les gridsearch lequel des quatres modèles est le plus performant
    # param_grid={"n_estimators":[1,25,50,100],
    #             "max_depth":[1,10,20,100],
    #             "min_samples_split":[2,4,8],
    #             "min_samples_leaf":[1,3,5,6],
    #
    #             }
    #
    # grid=GridSearchCV(RandomForestClassifier(),param_grid=param_grid,return_train_score=False,cv=5)
    # grid.fit(X_val,y_val)
    # # grid_rfc= grid.best_params_
    # # grid_rfc_score = grid.best_score_
    # grid_rfc_model = grid.best_estimator_
    #Logistic Regression

    # param_grid={"random_state":[1,42],
    #             "intercept_scaling":[1,2,4,5],
    #             "verbose":[0,1,5,7]
    #             }
    #
    # grid=GridSearchCV(LogisticRegression(),param_grid=param_grid,return_train_score=False,cv=5)
    # grid.fit(X_val,y_val)
    # # grid_lg= grid.best_params_
    # # grid_lg_score = grid.best_score_
    # grid_lg_model = grid.best_estimator_

    #DecisionTree Classifier

    # param_grid={"random_state":[None,1,42],
    #             "max_depth":[None,1,10,20,100],
    #             "min_samples_split":[2,4,8],
    #             "min_samples_leaf":[1,3,5,6]
    #             }
    # grid=GridSearchCV(DecisionTreeClassifier(),param_grid=param_grid,return_train_score=False,cv=5)
    # grid.fit(X_val,y_val)
    # # grid_dtc= grid.best_params_
    # # grid_dtc_score = grid.best_score_
    # grid_dtc_model = grid.best_estimator_


    #GradientBoostingClassifier

    param_grid={"random_state":[1,42],
                "n_estimators":[1,25,50],
                "max_depth":[1,10,20],
                "min_samples_split":[2,4],
                "min_samples_leaf":[1,3]
                }
    grid=GridSearchCV(GradientBoostingClassifier(),param_grid=param_grid,return_train_score=False,cv=5)
    grid.fit(X_val,y_val)
    #grid_gbc= grid.best_params_
    #grid_gbc_score = grid.best_score_
    grid_gbc_model = grid.best_estimator_
    # a = grid_gbc_model.score(X_test,y_test)
    # b = grid_dtc_model.score(X_test,y_test)
    # c = grid_lg_model.score(X_test,y_test)
    # d = grid_rfc_model.score(X_test,y_test)

    resultat['GradientBoosting'] = grid_gbc_model.score(X_test, y_test)

    # resultat['DecisionTree'] = grid_dtc_model.best_estimator_.score(X_test, y_test)
    # resultat['RandomForest'] = grid_rfc_model.best_estimator_.score(X_test, y_test)
    # resultat['LogisticRegression'] = grid_lg_model.best_estimator_.score(X_test, y_test)
    # for model, score in resultat.items():
    #     print(f"{model} Accuracy: {score}")

    return resultat,grid_gbc_model


def sauvegarde_modele(modele):
    with open("modèle.pkl", "wb") as fichier:
        modele= pickle.dump(modele,fichier)
    return modele


def chargement_modele():
    with open(r"modèle.pkl", "rb") as f:
        data = pickle.load(f)
    return data


def carte(data,pred):
    data2= data.copy()
    data2['predictions'] = pred
    figure = px.scatter_mapbox(data2,lon="longitude",
                               lat="latitude",
                               zoom=10,
                               mapbox_style="open-street-map",
                               color='fk_arb_etat'
    )
    figure.update_layout(mapbox_style="open-street-map")

    plo.plot(figure,filename="map.html")


def carte_prob(data, prob_pred):
    data2 = data.copy()
    data2['probabilite'] = prob_pred[:, 1]
    figure = px.scatter_mapbox(data2, lon="longitude", lat="latitude", zoom=10,
                               mapbox_style="open-street-map", color='probabilite',
                               color_continuous_scale=px.colors.sequential.Viridis)
    figure.update_layout(mapbox_style="open-street-map")
    plo.plot(figure, filename="map_prob.html")



In [5]:
def main():



    chemin= r"data_json"
    data_dic = data.to_dict()
    with open(chemin, 'w') as data_json:
        json.dump(data_dic, data_json)
    data2 = data.copy()
    X_2 = data2[new_data][donnes_quantitatives]
    data2['fk_arb_etat'] = data2['fk_arb_etat'].apply(lambda x: 'Non Essouché' if x != 'Essouché' else x)
    X_2 = std.transform(X_2)
    resultat, modele = gradient()
    y_pred = modele.predict(X_test)
    evaluation_modele(y_test, y_pred)
    plt.show()
    predictions = modele.predict(X_2)
    sauvegarde_modele(modele)
    carte(data, predictions)
    predictions = modele.predict_proba(X_2)
    carte_prob(data, predictions)
    data2 = pd.read_csv('Data_Arbre.csv').to_json()

    return resultat


In [ ]:
main()